*بخش اول:*
---
### 1_1
In Vision Transformer (ViT), images are tokenized by dividing them into smaller patches. Each image x with dimensions H×W×C (height, width, channels) is split into non-overlapping patches of size P×P (with stride = P). The number of patches is N = (H*W)/P^2, and each patch is flattened into a vector.

These flattened patches are then projected into a latent space of D (token vector). Then a class token is added to each patch tokens, also positional embedding is used for spatial imformation of patches. Finally, the patches are processed by the Transformer, and their classes are predicted.

In [ ]:
# 1_1 and 1_2
'''


FOR THIS QUESTION I DID NOT USE GPT


In this question, we want to tokenize the images and using tokenized images, # 1_1
train a image classifier. For this sake, we use the MNIST dataset, patch the 
images, cluster the patchs using kmeans and classify the tokens using a simple 
4gram.

We divide our proect into 2 parts:


Part 1: Tokenizing Images with KMeans:

    1) Load the MNIST dataset.

    2) Divide each image into 14x14 patchs (stride 14).

    3) Use KMeans clustering to assign each patch to one of 256 clusters (tokens).

    4) The result for each image will be a sequence of 16 tokens.


Part 2: Training a 4-Gram Classifier:

    1) Use the tokenized image sequences as input.

    2) Train a 4-gram model on these sequences, using a classifier like Logistic 
       Regression to predict the image class.

    3) Evaluate the model on the test dataset and show example predictions.

'''


# import necessary libraries
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


# Part 1: Loading MNIST dataset and tokenizing using KMeans


# load the MNIST dataset
mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target']

# normalize the images by deviding by 255
X = X/255.0

# reshape images to 28x28 format
X = X.to_numpy().reshape(-1, 28, 28)
y = y.astype(int)


# split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# function to extract patchs from an image
def extract_patchs(image, patch_size=7, stride=7):
    patchs = []
    for i in range(0, image.shape[0], stride):
        for j in range(0, image.shape[1], stride):
            patch = image[i:i+patch_size, j:j+patch_size]

            if patch.shape == (patch_size, patch_size):
                patchs.append(patch.flatten())

    return np.array(patchs)


# extract patchs from training images
patch_size, stride = 7, 7
patchs = []

for img in X_train:
    img_patchs = extract_patchs(img, patch_size, stride)
    patchs.append(img_patchs)

patchs = np.vstack(patchs)  # flatten all patchs into a single dataset for clustering


# apply KMeans for tokenizing the patchs
n_clusters = 256  # set the number of clusters (tokens)
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, 
                         max_iter=1000, batch_size=128)
kmeans.fit(patchs)


# tokenize the training dataset
def tokenize_image(image, patch_size=7, stride=7, 
                   clustering_model=KMeans(n_clusters=256, random_state=42)):
    img_patchs = extract_patchs(image, patch_size, stride)
    tokens = clustering_model.predict(img_patchs) # assign each patch to a cluster
    return tokens


# tokenize the data
tokenized_images_train = [tokenize_image(img, patch_size, stride, kmeans) 
                            for img in X_train]
tokenized_images_test = [tokenize_image(img, patch_size, stride, kmeans) 
                            for img in X_test]



c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:528: RuntimeWarning: invalid value encountered in cast
  fill_value = lib.item_from_zerodim(np.array(np.nan).astype(dtype))
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


*بخش دوم:*
---

In [ ]:
# # 2_1
# # Part 2: Training a 4-Gram Model


# # Function to generate 4-gram training data
# def generate_4gram_data(tokenized_images, y_labels, ngram_size=4):
#     X_ngrams = []
#     y_ngrams = []
    
#     for tokens, label in zip(tokenized_images, y_labels):
#         for i in range(len(tokens) - ngram_size + 1):
#             ngram = tokens[i:i + ngram_size]
#             X_ngrams.append(ngram)
#             y_ngrams.append(label)
    
#     return np.array(X_ngrams), np.array(y_ngrams)

# # Generate 4-gram data for training and test sets
# X_train_ngrams, y_train_ngrams = generate_4gram_data(tokenized_images_train, 
#                                                      y_train)
# X_test_ngrams, y_test_ngrams = generate_4gram_data(tokenized_images_test, y_test)

# # Train a logistic regression classifier on the 4-gram data
# clf = RandomForestClassifier()
# clf.fit(X_train_ngrams, y_train_ngrams)

# # Evaluate the model on the test set
# y_pred = clf.predict(X_test_ngrams)
# accuracy = accuracy_score(y_test_ngrams, y_pred)

# # Show the accuracy and some sample predictions
# print("Accuracy of 4-gram model:", accuracy)

# # 2_2
# print("Sample predictions:", y_pred[:10])
# print("True labels:", y_test_ngrams[:10])


class NGramClassifier:
    def __init__(self, n=4):
        self.n = n
        self.vocab = set()
        self.ngram_counts_per_class = defaultdict(Counter)
        self.total_ngrams_per_class = defaultdict(int)
        self.class_counts = Counter()
        self.classes = []

    def count_ngrams(self, tokens):
        """Count the n-grams in the given tokens."""
        ngrams = Counter([tuple(tokens[i:i+self.n]) for i in range(len(tokens)-self.n+1)])
        return ngrams

    def fit(self, tokenized_images, labels):
        """Train the n-gram classifier by counting n-grams per class."""
        self.classes = list(set(labels))  # Store unique classes
        for tokens, label in zip(tokenized_images, labels):
            ngrams = self.count_ngrams(tokens)
            self.ngram_counts_per_class[label].update(ngrams)
            self.total_ngrams_per_class[label] += sum(ngrams.values())
            self.vocab.update(ngrams)
            self.class_counts[label] += 1

    def calculate_class_prob(self, ngrams, class_label):
        """Calculate the log probability of the class given the n-grams."""
        total_ngrams = self.total_ngrams_per_class[class_label]
        class_prob = np.log(self.class_counts[class_label] / sum(self.class_counts.values()))  # P(class)

        log_prob = class_prob  # Start with prior
        for ngram, count in ngrams.items():
            # Use Laplace smoothing for unseen n-grams
            ngram_count = self.ngram_counts_per_class[class_label].get(ngram, 0) + 1
            prob = ngram_count / (total_ngrams + len(self.vocab))  # P(ngram | class)
            log_prob += count * np.log(prob)  # log(P(ngram | class)) * count

        return log_prob

    def predict(self, tokenized_images):
        """Predict the class for each input image based on n-grams."""
        predictions = []
        for tokens in tokenized_images:
            ngrams = self.count_ngrams(tokens)
            class_probs = {cls: self.calculate_class_prob(ngrams, cls) for cls in self.classes}
            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)

        return predictions

    def evaluate(self, tokenized_images, labels):
        """Evaluate the classifier on a test set."""
        predictions = self.predict(tokenized_images)
        accuracy = np.mean(np.array(predictions) == np.array(labels))
        return accuracy


# Train and evaluate the NGram classifier
ngram_classifier = NGramClassifier(n=4)
ngram_classifier.fit(tokenized_images_train, y_train)

# Evaluate on the test set
accuracy = ngram_classifier.evaluate(tokenized_images_test, y_test)
print(f"Accuracy of 4-gram classifier: {accuracy:.2f}")

Accuracy of 4-gram classifier: 0.79
